# Private-IP-Location-Mapping
## Private IPv4 to Geo-Location mapping for Dynatrace Appmon 

In [1]:
import pandas as pd
import numpy as np
import radix as rx


In [2]:
#Read validation table first
valid_table = pd.read_csv('InputOutput/ValidationTable_v3.csv',sep=';')
valid_table.head()
print("Number of City-Region-Country in validation table is = "+str(len(valid_table)))

Number of City-Region-Country in validation table is = 356


In [3]:
#Read GNOC_Locations.csv, remove useless columns
# Remove entries with empty Country names
# Remove entries with empty city names
# Full OUTER MERGE the table with Validation to create IP-Country-Region-City-Mapping = ICRC_Mapping
# In ICRC Mappinng, remove entries where Country names do not match, and check same city name in two countries
# RUN the RADIX TABLE ON ICRC_Mapping to create ICRC_Radix Table
# Review Radix Table for valid city mappings > Final Table
# Use final Table. This Radix Table must be reversed sorted.


In [4]:
# Read the GNOC Location csv

gnoc_locs = pd.read_csv('InputOutput/GNOC_Locations.csv', sep=';')

gnoc_locs.head()

len(gnoc_locs)

# Drop useless columns in gnoc table

gnoc_locs.columns


gnoc_locs.drop(['Site', 'Region','Building', 'Location',
       'File_Name', 'Support_Group', 'IP Address Type', 'ATON_Building_ID',
       'CRES_Building_ID', 'comments'],axis=1, inplace=True)

gnoc_locs.tail()

# Change Column Name network -> Network

gnoc_locs.rename(columns = {'network':'Network'}, inplace = True)

gnoc_locs.tail()


# Remove the Rows which have EMPTY city location 


a = gnoc_locs[gnoc_locs['City'] == '']

b = pd.isna(gnoc_locs['City'])





In [5]:
gnoc_locs.head()

,Network,City,Country
0,10.100.131.0/27,Mumbai,India
1,10.100.131.160/27,Mumbai,India
2,10.100.132.0/22,Mumbai,India
3,10.100.136.0/23,Mumbai,India
4,10.100.138.0/23,Mumbai,India


In [6]:
# Merge with Validation table to yeild location with region

val_tbl = pd.read_csv('InputOutput/ValidationTable_v3.csv', sep=';')

val_tbl.head()




,Country,Region,City
0,Argentina,Buenos Aires,Buenos Aires
1,Argentina,Buenos Aires,San Telmo
2,Argentina,Buenos Aires,FW-unknown
3,Australia,Australian Capital Territory,Canberra
4,Australia,New South Wales,Sydney


In [7]:

# Full outer join

full_ = gnoc_locs.merge(val_tbl, on='City', how='outer')

# left join with ip table on left

left_ = gnoc_locs.merge(val_tbl, on='City', how='left')

pd.set_option('display.max_rows', 5000)

full_.tail(5)

full_.to_csv("fulljoinip.csv", sep=";")


# the full join do below
# discard subnets with NAN or empty
# country y is from mapping, discard rows which have NAN
# mapping must be correct
# how to handle the city name with 2 countties. in this case country x which is original will take precedence

# Empty region, must be replaced by __NO_REGION__



#### After this it needs intelligent filtering:

Remove NaN in city and network columns
Remove or correct when countries do kot match





### NEED TO FIND DUPLICATE ENTRIES and RESOLVE CONFLICTS MANUALLY

In [8]:



# Put the subnets in Radix tree

rt = rx.Radix()
mapp=left_
for i in range(0, len(mapp)):
    subnet    = str(mapp.iloc[i]['Network']).rstrip('\n').lstrip('\n')
    country_x = str(mapp.iloc[i]['Country_x']).rstrip('\n').lstrip('\n')
    country_y = str(mapp.iloc[i]['Country_y']).rstrip('\n').lstrip('\n')
    region    = str(mapp.iloc[i]['Region']).rstrip('\n').lstrip('\n')
    city = str(gnoc_locs.iloc[i]['City']).rstrip('\n').lstrip('\n')
    if (city!="NaN" and subnet!="NaN" and subnet!=""):
        rn = rt.add(subnet)
        rn.data["Region"] = region
        rn.data["City"] = city
        rn.data["Country_x"] = country_x
        rn.data["Country_y"] = country_y


                        
# write the radix tree in file

outp = open("InputOutput/out.csv","w")


nodes = rt.nodes()
for rnode in nodes:
    outp.write(str(rnode.prefix)+";"+str(rnode.data["City"])+";"+str(rnode.data["Country_x"])+";"+str(rnode.data["Country_y"])+";"+str(rnode.data["Region"])+"\n")
             






IndexError: single positional indexer is out-of-bounds